In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow import keras
from keras.layers import Input, Dense, Dropout, LeakyReLU
from keras.models import Model, Sequential
from keras.datasets import mnist
# from keras.optimizers import Adam
from keras.optimizers import Adam
from keras import initializers

In [3]:
np.random.seed(10)
random_dim=100

In [5]:
def load_minst_data():
    # load the data
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    # normalize our inputs to be in the range[-1, 1]
    x_train = (x_train.astype(np.float32) - 127.5)/127.5
    # convert x_train with a shape of (60000, 28, 28) to (60000, 784)
    # so we have 784 columns per row
    x_train = x_train.reshape(60000, 784)
    return (x_train, y_train, x_test, y_test)

In [6]:
def get_optimizer():
    return Adam(learning_rate=0.0002, beta_1=0.5)

In [7]:
def get_generator(optimizer):
    generator = Sequential()
    generator.add(Dense(256, input_dim=random_dim, \
            kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(784, activation='tanh'))
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return generator

In [8]:
def get_discriminator(optimizer):

    discriminator = Sequential()

    discriminator.add(Dense(1024, input_dim=784, \

                kernel_initializer=initializers.RandomNormal(stddev=0.02)))

    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(512))

    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(256))

    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(1, activation='sigmoid'))

    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)

    return discriminator






In [10]:
def get_gan_network(discriminator, random_dim, generator, optimizer):
    # We initially set trainable to False since we only want to train either the
    # generator or discriminator at a time
    discriminator.trainable = False
    # gan input (noise) will be 100-dimensional vectors
    gan_input = Input(shape=(random_dim,))
    # the output of the generator (an image)
    x = generator(gan_input)
    # get the output of the discriminator (probability if the image is real or not)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer)
    return gan

In [12]:
def plot_generated_images(epoch, generator, examples=100, dim=(10, 10), \
                          figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', \
                   cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

In [13]:
def train(epochs=1, batch_size=128):
    #1 Get the training and testing data
    x_train, y_train, x_test, y_test = load_minst_data()
    # Split the training data into batches of size 128
    batch_count = x_train.shape[0] / batch_size
    #2. Build our GAN netowrk
    adam = get_optimizer()
    generator = get_generator(adam)
    discriminator = get_discriminator(adam)
    gan = get_gan_network(discriminator, random_dim, generator, adam)

In [20]:
import numpy as np
from tqdm import tqdm

def train_gan(generator, discriminator, gan, x_train, epochs, batch_size, random_dim, plot_generated_images):
    batch_count = x_train.shape[0] // batch_size

    for epoch in range(1, epochs + 1):
        print('-' * 15, f'Epoch {epoch}', '-' * 15)

        for _ in tqdm(range(batch_count)):
            # Generate a batch of random noise
            noise = np.random.normal(0, 1, size=(batch_size, random_dim))

            # Select a random batch of real images
            idx = np.random.randint(0, x_train.shape[0], batch_size)
            image_batch = x_train[idx]

            # Generate fake images
            generated_images = generator.predict(noise)

            # Combine real and fake images
            X = np.concatenate([image_batch, generated_images])

            # Create labels: real images labeled 0.9 (label smoothing), fake images labeled 0
            y_dis = np.zeros(2 * batch_size)
            y_dis[:batch_size] = 0.9

            # Train discriminator
            discriminator.trainable = True
            d_loss = discriminator.train_on_batch(X, y_dis)

            # Train generator
            noise = np.random.normal(0, 1, size=(batch_size, random_dim))
            y_gen = np.ones(batch_size)  # All fake images should be classified as real
            discriminator.trainable = False
            g_loss = gan.train_on_batch(noise, y_gen)

        # Log training progress
        print(f'Epoch {epoch}: D Loss = {d_loss:.4f}, G Loss = {g_loss:.4f}')

        # Save generated images every 20 epochs
        if epoch == 1 or epoch % 20 == 0:
            plot_generated_images(epoch, generator)

In [17]:
train(20, 128)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
